In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import sys
os.chdir('drive/My Drive/Colab Notebooks')

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 64
n_batch = mnist.train.num_examples // batch_size
# print(n_batch, mnist.train.num_examples)    # 859 55000

W0825 03:25:01.059711 140037632825216 deprecation.py:323] From <ipython-input-4-117f2a6e5922>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 03:25:01.062004 140037632825216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 03:25:01.063943 140037632825216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0825 03:25:02.036031 140037632825216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST_data/train-labels-idx1-ubyte.gz


W0825 03:25:02.362365 140037632825216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0825 03:25:02.871690 140037632825216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# 初始化权值
def weight_variable(shape):
    w = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(w)

# 初始化bias
def bias_variable(shape):
    bias = tf.zeros(shape) + 0.1
    return tf.Variable(bias)

# 卷积层
def conv2d(x, w):
    '''
    tf.nn.con2d(): 
    w = filter: [k_height, k_width, in_channel, out_channel]
    strides: [1, x_strides, y_strides, 1] 两端固定为1，x_strides为x方向的步长，y_strides为y方向的步长 
    '''
    return tf.nn.conv2d(x, filter=w, strides=[1,1,1,1], padding='SAME')

# 池化层
def max_pool_2x2(x):
    '''
    ksize: [1, x , y, 1]
    '''
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# 建立一个网络框架
## input[batch, in_height, in_width, in_channels] ----> conv1[batch, 28, 28, out_channels1 ]
## ----> pooling1[batch, 14, 14, out_channels1] ----> conv2[ ] ---->pooling2[ ]

# 把x转换为4维格式
x_image = tf.reshape(x, [-1,28,28,1])    # -1会自动匹配上批次的大小，因为CNN要求传入4维的数据

# 初始化第一层的权重和偏置
w_conv1 = weight_variable([5,5,1,32])    # 5*5的kernel，输入通道数是1，输出通道数是32
b_conv1 = bias_variable([32])

# 把x_image和权值向量进行卷积，再加上bias，然后使用relu激活函数，最后池化，完成第一层
a_conv1 = tf.nn.relu(conv2d(x_image, w_conv1)+b_conv1)
a_pool1 = max_pool_2x2(a_conv1)

# 初始化第二层的权重和偏置
w_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

a_conv2 = tf.nn.relu(conv2d(a_pool1, w_conv2)+b_conv2)
a_pool2 = max_pool_2x2(a_conv2)
## [64,7,7,64] 得到64个7*7*64的数据
## 把a_pool2扁平化处理，以便后续全连接层使用
a_pool2_flat = tf.reshape(a_pool2, shape=[-1, 7*7*64])

# 初始化第一个全连接层的权重和偏置
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

a_fc1 = tf.nn.relu(tf.matmul(a_pool2_flat, w_fc1)+b_fc1)
keep_prob = tf.placeholder(tf.float32)
a_fc1_drop = tf.nn.dropout(a_fc1, keep_prob)

# 初始化第二个全连接层
w_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

pred = tf.nn.softmax(tf.matmul(a_fc1_drop, w_fc2)+b_fc2)

loss = tf.losses.softmax_cross_entropy(y, pred)
train = tf.train.AdamOptimizer(0.0001).minimize(loss)

corret_pred = tf.equal(tf.argmax(y,1), tf.argmax(pred,1))
accuracy = tf.reduce_mean(tf.cast(corret_pred, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
#             print('{}-{}'.format(epoch, batch),end=' ')
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print('Epoch {}, test accuracy is {}'.format(epoch,acc))


W0825 03:25:22.233190 140037632825216 deprecation.py:506] From <ipython-input-5-c7adb59f2258>:61: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0825 03:25:22.308038 140037632825216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 0, test accuracy is 0.8664000034332275
Epoch 1, test accuracy is 0.8790000081062317
Epoch 2, test accuracy is 0.9739000201225281
Epoch 3, test accuracy is 0.9794999957084656
Epoch 4, test accuracy is 0.9783999919891357
Epoch 5, test accuracy is 0.9846000075340271
Epoch 6, test accuracy is 0.982200026512146
Epoch 7, test accuracy is 0.9869999885559082
Epoch 8, test accuracy is 0.9866999983787537
Epoch 9, test accuracy is 0.9883999824523926
Epoch 10, test accuracy is 0.9887999892234802
Epoch 11, test accuracy is 0.9890000224113464
Epoch 12, test accuracy is 0.9896000027656555
Epoch 13, test accuracy is 0.9872999787330627
Epoch 14, test accuracy is 0.9905999898910522
Epoch 15, test accuracy is 0.9876000285148621
Epoch 16, test accuracy is 0.9869999885559082
Epoch 17, test accuracy is 0.9898999929428101
Epoch 18, test accuracy is 0.9900000095367432
Epoch 19, test accuracy is 0.9894000291824341
Epoch 20, test accuracy is 0.991100013256073
